<a href="https://colab.research.google.com/github/muhammadhafiz27/Deep-Learning/blob/main/Deep_Belief_Network_(DBN)_dan_Capsule_Network_(CapsNet).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Implementasi Deep Belief Network (DBN)**

## **Import Library**

In [1]:
from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import MinMaxScaler
import numpy as np

## **Load dataset MNIST**

In [2]:
X, y = fetch_openml('Fashion-MNIST', version=1, return_X_y=True, as_frame=False)
X = X.astype('float32')
X = MinMaxScaler().fit_transform(X)
y = y.astype('int')

## **Definisikan RBM dan Logistic Regression**

In [3]:
rbm = BernoulliRBM(n_components=256, learning_rate=0.06, batch_size=10, n_iter=10, verbose=1)
logistic = LogisticRegression(max_iter=1000, solver='lbfgs', multi_class='multinomial')

## **DBN sebagai pipeline RBM + Logistic Regression**

In [4]:
dbn = Pipeline(steps=[('rbm', rbm), ('logistic', logistic)])
dbn.fit(X[:60000], y[:60000])

[BernoulliRBM] Iteration 1, pseudo-likelihood = -218.09, time = 22.59s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -216.79, time = 24.86s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -214.62, time = 25.09s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -205.87, time = 23.43s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -202.66, time = 22.42s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -208.80, time = 24.27s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -206.20, time = 23.85s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -208.27, time = 24.38s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -209.54, time = 23.83s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -199.73, time = 21.97s


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Pipeline(steps=[('rbm', BernoulliRBM(learning_rate=0.06, verbose=1)),
                ('logistic',
                 LogisticRegression(max_iter=1000, multi_class='multinomial'))])

## **Evaluasi**

In [5]:
score = dbn.score(X[60000:], y[60000:])
print(f"Akurasi DBN pada data uji: {score * 100:.2f}%")

Akurasi DBN pada data uji: 81.60%


# **Implementasi Capsule Network (CapsNet)**

## **Import Library**

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

## **Load dataset**

In [7]:
transform = transforms.Compose([transforms.ToTensor()])
train_loader_fm = DataLoader(
    datasets.FashionMNIST('./data', train=True, download=True, transform=transform),
    batch_size=64,
    shuffle=True
)
test_loader_fm = DataLoader(
    datasets.FashionMNIST('./data', train=False, transform=transform),
    batch_size=64,
    shuffle=False
)

100%|██████████| 26.4M/26.4M [00:02<00:00, 10.5MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 200kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.73MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 14.0MB/s]


## **Definisi CapsNet sederhana**

In [11]:
class CapsuleLayer(nn.Module):
    def __init__(self, num_capsules, num_route_nodes, in_channels, out_channels, num_iterations=3):
        super(CapsuleLayer, self).__init__()
        self.num_route_nodes = num_route_nodes
        self.num_capsules = num_capsules
        self.num_iterations = num_iterations
        self.route_weights = nn.Parameter(torch.randn(num_capsules, num_route_nodes, in_channels, out_channels))

    def squash(self, tensor, dim=-1):
        norm = (tensor ** 2).sum(dim=dim, keepdim=True)
        scale = norm / (1 + norm)
        return scale * tensor / torch.sqrt(norm + 1e-8)

    def forward(self, x):
        # x input after primary_caps and view: (batch_size, num_route_nodes, in_channels)

        # Correctly align dimensions for batch matrix multiplication
        # x_expanded: (batch_size, 1, num_route_nodes, 1, in_channels)
        # W_expanded: (1, num_capsules, num_route_nodes, in_channels, out_channels)
        # Result after @ : (batch_size, num_capsules, num_route_nodes, 1, out_channels)
        x = (x.unsqueeze(1).unsqueeze(-2) @ self.route_weights.unsqueeze(0))

        # Removes the singleton dimension at index 3, resulting in (B, D, P, C_out)
        # Example: (batch_size, num_capsules, num_route_nodes, out_channels) e.g., (64, 10, 1152, 16)
        x = x.squeeze(3)

        # b has shape (batch_size, num_capsules, num_route_nodes) e.g., (64, 10, 1152)
        b = torch.zeros_like(x[:, :, :, 0], device=x.device)

        for i in range(self.num_iterations):
            # c has shape (batch_size, num_capsules, num_route_nodes) e.g., (64, 10, 1152)
            c = F.softmax(b, dim=2)

            # Previous error: c.unsqueeze(4) was out of range for a 3D tensor.
            # Fix: unsqueeze at the last dimension (-1 or 3) to enable broadcasting with x.
            # c.unsqueeze(-1) -> (batch_size, num_capsules, num_route_nodes, 1)
            # (c.unsqueeze(-1) * x) -> (batch_size, num_capsules, num_route_nodes, out_channels)
            # .sum(dim=2) sums over num_route_nodes, resulting in (batch_size, num_capsules, out_channels)
            s = (c.unsqueeze(-1) * x).sum(dim=2)
            v = self.squash(s)

            if i < self.num_iterations - 1:
                # v[:, :, None, :] -> (batch_size, num_capsules, 1, out_channels)
                # (x * v[:, :, None, :]) -> (batch_size, num_capsules, num_route_nodes, out_channels)
                # .sum(dim=-1) sums over out_channels, resulting in (batch_size, num_capsules, num_route_nodes)
                b = b + (x * v[:, :, None, :]).sum(dim=-1)
        return v

# Re-initialize model after class definition change for the fix to take effect
model = CapsNet()
print(model)

CapsNet(
  (conv1): Conv2d(1, 256, kernel_size=(9, 9), stride=(1, 1))
  (primary_caps): Conv2d(256, 256, kernel_size=(9, 9), stride=(2, 2), groups=8)
  (digit_caps): CapsuleLayer()
)


## **Definisi Loss Function (Margin Loss) dan Optimizer**

In [12]:
class MarginLoss(nn.Module):
    def __init__(self, m_plus=0.9, m_minus=0.1, lambda_val=0.5):
        super(MarginLoss, self).__init__()
        self.m_plus = m_plus
        self.m_minus = m_minus
        self.lambda_val = lambda_val

    def forward(self, lengths, target):
        # lengths adalah panjang vektor output (dari CapsNet forward)
        # target adalah label kelas (0, 1, ..., 9)

        # T_k (Indicator function): 1 untuk kelas target, 0 untuk non-target
        T_k = torch.zeros(lengths.size()).to(lengths.device)
        T_k.scatter_(1, target.unsqueeze(1), 1)

        # Loss untuk kelas target: L_k = T_k * max(0, m+ - ||vk||)^2
        loss_pos = T_k * F.relu(self.m_plus - lengths).pow(2)

        # Loss untuk kelas non-target: L_k = lambda * (1 - T_k) * max(0, ||vk|| - m-)^2
        loss_neg = self.lambda_val * (1.0 - T_k) * F.relu(lengths - self.m_minus).pow(2)

        # Total Loss adalah penjumlahan loss di seluruh kapsul
        total_loss = loss_pos + loss_neg
        return total_loss.sum(dim=1).mean()

# Inisialisasi Model, Loss, dan Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CapsNet().to(device)
criterion = MarginLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

## **Training Loop untuk CapsNet**

In [15]:
# 4. Training Loop
epochs = 10
print(f"\nMemulai Pelatihan CapsNet pada {device}...")

for epoch in range(1, epochs + 1):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(train_loader_fm):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data) # Output adalah panjang vektor
        loss = criterion(output, target)

        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        # Akurasi: Ambil indeks kapsul dengan panjang vektor terbesar
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

        if batch_idx % 100 == 0:
            print(f'Epoch: {epoch} [{batch_idx*len(data)}/{len(train_loader_fm.dataset)}] Loss: {loss.item():.6f}')

    avg_train_loss = train_loss / len(train_loader_fm)
    train_accuracy = 100. * correct / total

    # Evaluasi (Test)
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader_fm:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    avg_test_loss = test_loss / len(test_loader_fm)
    test_accuracy = 100. * correct / total

    # Hasil
    print(f'\n--- Epoch {epoch}/{epochs} Selesai ---')
    print(f'Loss Pelatihan: {avg_train_loss:.4f}, Akurasi Pelatihan: {train_accuracy:.2f}%')
    print(f'Loss Uji: {avg_test_loss:.4f}, Akurasi Uji: {test_accuracy:.2f}%')
    print('--------------------------------\n')


Memulai Pelatihan CapsNet pada cuda...
Epoch: 1 [0/60000] Loss: 0.805927
Epoch: 1 [6400/60000] Loss: 0.180763
Epoch: 1 [12800/60000] Loss: 0.138542
Epoch: 1 [19200/60000] Loss: 0.112778
Epoch: 1 [25600/60000] Loss: 0.094466
Epoch: 1 [32000/60000] Loss: 0.121969
Epoch: 1 [38400/60000] Loss: 0.107371
Epoch: 1 [44800/60000] Loss: 0.098057
Epoch: 1 [51200/60000] Loss: 0.074984
Epoch: 1 [57600/60000] Loss: 0.139418

--- Epoch 1/10 Selesai ---
Loss Pelatihan: 0.1357, Akurasi Pelatihan: 82.92%
Loss Uji: 0.1038, Akurasi Uji: 86.08%
--------------------------------

Epoch: 2 [0/60000] Loss: 0.067716
Epoch: 2 [6400/60000] Loss: 0.094368
Epoch: 2 [12800/60000] Loss: 0.102371
Epoch: 2 [19200/60000] Loss: 0.057720
Epoch: 2 [25600/60000] Loss: 0.064154
Epoch: 2 [32000/60000] Loss: 0.128709
Epoch: 2 [38400/60000] Loss: 0.088498
Epoch: 2 [44800/60000] Loss: 0.111047
Epoch: 2 [51200/60000] Loss: 0.123216
Epoch: 2 [57600/60000] Loss: 0.081712

--- Epoch 2/10 Selesai ---
Loss Pelatihan: 0.0916, Akurasi 